In [3]:
import cv2
from gaze_tracking import GazeTracking
import pandas as pd
import datetime
import numpy as np

def get_data():
    l=[]
    center_left=[]
    center_right=[]
    l_x=[]
    l_y=[]
    r_x=[]
    r_y=[]
    d={'Time':[],'Left eye':[],'Right eye':[]}

    gaze = GazeTracking()
    webcam = cv2.VideoCapture(0)

    while True:
        # We get a new frame from the webcam
        _, frame = webcam.read()

        # We send this frame to GazeTracking to analyze it
        gaze.refresh(frame)

        frame = gaze.annotated_frame()
        text = ""

        if gaze.is_blinking():
            text = "Blinking"
            l.append(datetime.datetime.now())
        elif gaze.is_left():
            text = "Looking left"
        elif gaze.is_center():
            text = "Looking right"
            

        cv2.putText(frame, text, (90, 60), cv2.FONT_HERSHEY_DUPLEX, 1.6, (147, 58, 31), 2)

        left_pupil = gaze.pupil_left_coords()
        right_pupil = gaze.pupil_right_coords()
        try:
            l_x.append(gaze.pupil_left_coords()[0])
            l_y.append(gaze.pupil_left_coords()[1])
            r_x.append(gaze.pupil_right_coords()[0])
            r_y.append(gaze.pupil_right_coords()[1])
        except:            
            l_x.append(0)
            l_y.append(0)
            r_x.append(0)
            r_y.append(0)
        cv2.putText(frame, "Left pupil:  " + str(left_pupil), (90, 130), cv2.FONT_HERSHEY_DUPLEX, 0.9, (147, 58, 31), 1)
        cv2.putText(frame, "Right pupil: " + str(right_pupil), (90, 165), cv2.FONT_HERSHEY_DUPLEX, 0.9, (147, 58, 31), 1)
        #print((left_pupil,right_pupil))
        try:
            d['Left eye'].append((left_pupil[0],left_pupil[1]))
            d['Right eye'].append((right_pupil[0],right_pupil[1]))
            d['Time'].append(datetime.datetime.now())
        except:
            d['Left eye'].append(0)
            d['Right eye'].append(0)
            d['Time'].append(datetime.datetime.now())

        cv2.imshow("Frame", frame)

        if cv2.waitKey(1) == 27:
            break


    eye_coordinates=pd.DataFrame(d)
    eye_coordinates.columns=['Time','Left eye','Right eye']
    eye_blinking=pd.Series(l)
    return eye_coordinates#,eye_blinking,center_right,center_left,l_x,l_y,r_x,r_y

both=get_data()


In [2]:
"""import pickle
q=open('right_eye_center2.pkl','wb')
pickle.dump(d,q)
q.close()"""

"import pickle\nq=open('right_eye_center2.pkl','wb')\npickle.dump(d,q)\nq.close()"

In [3]:
"""import pickle
file1=open('both_eye2.pkl','rb')
file2=open('left_eye_center2.pkl','rb')
file3=open('right_eye_center2.pkl','rb')
both=pickle.load(file1)
l_c=pickle.load(file2)
r_c=pickle.load(file3)
file1.close()
file2.close()
file3.close()
#both,l_c,r_c"""



"import pickle\nfile1=open('both_eye2.pkl','rb')\nfile2=open('left_eye_center2.pkl','rb')\nfile3=open('right_eye_center2.pkl','rb')\nboth=pickle.load(file1)\nl_c=pickle.load(file2)\nr_c=pickle.load(file3)\nfile1.close()\nfile2.close()\nfile3.close()\n#both,l_c,r_c"

In [4]:
left=both['Left eye']
right=both['Right eye']
l_x=[x[0] if x!=0 else 0 for x in left]
l_y=[x[1] if x!=0 else 0 for x in left]
r_x=[x[0] if x!=0 else 0 for x in right]
r_y=[x[1] if x!=0 else 0 for x in right]

#left,right,l_x,l_y,r_x,r_y

In [5]:
sec=[]
for i in both['Time']:
    sec.append(float(i.strftime('%S.%f')))
#sec

In [6]:
both['Time'].iloc[len(both)-1]-both['Time'].iloc[0]
#(len(both)/284.45)*60

Timedelta('0 days 00:02:54.344111')

In [7]:
l_c_x=[x[0] if x!=0 else 0 for x in left]
l_c_y=[x[1] if x!=0 else 0 for x in left]
#len(l_c_y)

In [8]:
import numpy as np
def removeOutliers(x, outlierConstant):
    a = np.array(x)
    upper_quartile = np.percentile(a, 75)
    lower_quartile = np.percentile(a, 25)
    IQR = (upper_quartile - lower_quartile) * outlierConstant
    quartileSet = (lower_quartile - IQR, upper_quartile + IQR)
    resultList = []
    for y in a.tolist():
        if(y>=quartileSet[0] and y<=quartileSet[1]):
            resultList.append(y)
    return resultList
l_c_x=removeOutliers(l_c_x,1.5)
l_c_y=removeOutliers(l_c_y,1.5)
l_x=[sum(removeOutliers(l_x,1.5))/len(removeOutliers(l_y,1.5)) if x==0 else x for x in l_x]
l_y=[sum(removeOutliers(l_y,1.5))/len(removeOutliers(l_y,1.5)) if x==0 else x for x in l_y]
#len(l_c_y)

In [9]:
c1=sum(l_c_x)/len(l_c_x)
c2=sum(l_c_y)/len(l_c_y)
#c1,c2
#left

In [10]:
def distract(l_y,c2):
    deltas=[[]]
    sec=[]
    per=[]
    status=0
    for i in range(len(l_y)):
        if(l_y[i]<=(c2-5) or l_y[i]>=(c2+5)):
            if status==0:
                deltas.append([])
                status=1
            deltas[len(deltas)-1].append(both['Time'].iloc[i])
        else:
            status=0
    for i in deltas[1:]:
        sec.append(i[len(i)-1]-i[0])
    for i in deltas[1:]:
        per.append(len(i))
    return sec,per
deltas_x,per_x=distract(l_x,c1)
deltas_y,per_y=distract(l_y,c2)
deltas_x=[i.total_seconds()*60 for i in deltas_x]
deltas_y=[i.total_seconds()*60 for i in deltas_y]
#per_x

In [11]:
def score(deltas_x,deltas_y,per_x,per_y):
    s1,s2,s3,s4=0,0,0,0
    for i in deltas_x:
        if i!=0:
            if(i<70):
                s1+=((0.2*i)/70)
            elif(i>=70 and i<=100):
                s1+=(0.2+(((i-70)*0.4)/30))
            elif(i>100):
                s1+=(0.6+(((i-100)*0.4)/50))
    s1=s1/len(deltas_x)
    for i in deltas_y:
        if i!=0:
            if(i<70):
                s2+=((0.2*i)/70)
            elif(i>=70 and i<=100):
                s2+=(0.2+(((i-70)*0.4)/30))
            elif(i>100):
                s2+=(0.6+(((i-100)*0.4)/50))
    s2=s2/len(deltas_y)
    for i in per_x:
        if(i>=15 and i<=60):
            s3+=((i-15)/45)
        if i>60:
            s3+=1
    s3=s3/len(per_x)
    for i in per_y:
        if(i>=15 and i<=60):
            s4+=((i-15)/45)
        if i>60:
            s4+=1
    s4=s4/len(per_y)
    return s1,s2,s3*12,s4*12
final_score=sum(score(deltas_x,deltas_y,per_x,per_y))/4

final_score
            
            
    

0.33216474869047613